# Spark 

Spark is a big-data processing framework that I describe as follows: 

1. We have a lot of data.
2. We will split our data into chunks, called "partitions".
3. We read data in partitions, we write data in partitions, and all the processing in between, happens in partitions. 
4. Given that data is in partitions, each partition can be worked on in parallel, in serial, or a combination of both. 
5. Much of the "behind-the-scenes" magic of Spark consists of scheduling the work of processing each partition to a different "worker", where workers are potentially spread across machines.
6. Much of the day-to-day work of a Spark programmer consists of trying to compute what you want to compute, given the constraint that your data is all split up into a bunch of f(*&#@ing partitions.

Spark is succesful because this framework of splitting data up into partitions is a very useful abstraction that's extremely _flexible_. You can work with very very large data, do basically anything you want with the data, and do so across any number of machines. That's very powerful! 

But computing with big data efficiently requires the programmer to consider the underlying data model of partitions. Sometimes we need data to be partitioned in a different way then it comes. Spark can shuffle data between partitions in this instance, but "shuffling" is an expensive operation. The goal is always to do everything you want while avoiding the need to shuffle data between partitions as much as possible. 

We already know one basic pattern that is super efficient in this framework: Map + Reduce! 

By it's very definition, map works on each partition (indeed, one each "row" or element individually). Reduce is an operation that "folds" each element of the data into an "accumulator" via a given operation. If the reduce operation is commutative and associative (like `sum`), it can be done first within a partition and then the results from each partition combined via the same function across partitions. This is the simplest way to "aggregate" our data into a result.

In [ ]:
from pyspark.sql import SparkSession

# We start by creating what is called a "SparkSession":
spark = SparkSession \
    .builder \
    .appName("PySpark Intro") \
    .getOrCreate()

In [ ]:
# Spark lets us read many file types, including JSON. 
# Reading, like most operations in Spark, is lazy,
# so tihs operation won't read the data into memory,
# it will just scan the data to learn the "schema".
orders = spark.read.json('data/orders.json')

# orders is an instance of a Spark DataFrame. 
# .printSchema is a method that can be used to
# see that "schema" of the DataFrame. Let's take
# a look at what we mean: 

orders.printSchema()

# DataFrames + RDDs

The RDD (Resilient Distributed Dataset) is the fundamental data structure of Spark. You can think of it as a partitioned list!

DataFrame's are a higher-level construct built on top of RDDs. They have "schemas" which means that each column has a "type" (ie string, int, etc.). Unlike our basic, old-school SQL database, columns can be nested! Because the columns have types, Spark can optimize operations in a Dataframe. In general, they allow us to easily write SQL-style operations on our data with super-optimized execution under-the-hood. This is great for a large set of analytics use-cases!

The basic rule of thumb is: use DataFrames if you can fit what you're trying to do within their API, because it will usually be faster, otherwise use the RDDs directly if you need the flexibility. 

For this tutorial, we will be focusing on RDDs, because it's educational and simple to get started with.


In [ ]:
# We can access the RDD of a DataFrame to perform simple operations on the
# data with basic Python functions.

# Let's take a look at the RDD:

orders.rdd

In [ ]:
# Remember: all operations in Spark are lazy. So we haven't actually
# read the data from the file yet.

# We can take a look at the first few elements in the data with the .take
# method: 

orders.rdd.take(2)

In [ ]:
# Notice that .take returns a list of elements. As we said, 
# an RDD is like a distributed, lazy, partitioned list. When we
# run .take, we bring all the elements into memory into a list.

# What are the elements? Well this row was created from a DataFrame,
# so all of the elements are instance of the Spark Row class!

# Row's in Spark are a lot like Row's in pandas. We can access the 
# individual values with "dot notation":

for r in orders.rdd.take(2):
    print(r.customer.country)

In [ ]:
# .collect is like .take, but it brings ALL the data into a list
# If you try and collect too much data, your memory will blow up!

# Note some things here:

# 1. RDDs have a method called .filter
# 2. RDDs let us use basic Python functions to do all the work!
# 3. We can read about the RDD methods here: 
#    https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD

orders.rdd.filter(lambda r: r.customer.country == 'Belgium').collect()

In [ ]:
# Exercise 1

# Let's try and repeat the operation from the Python map-reduce tutorial, 
# of finding the total sales, but now with the spark RDD. 
# You will need to read the Spark Documentation and find the following functions: map, reduce
# which are available as methods directly on the RDD!



In [ ]:
# Exercise 2

# Take a look at the RDD method .flatMap
# Now try and use that method to rewrite your previous function!


In [ ]:
# Exercise 3

# RDD's have a concept of "key, value" which
# Is implemented simply as a Tuple: (k,v). 
# So if we make an RDD of tuples, we have some
# special methods that will operate on them, 
# such as .mapValues or .reduceByKey

# Get the total sales by country, by first mapping 
# your RDD into a tuple (k,v) where the Key is the 
# country, then reducing by summing the total sales. 
# You can use .collectAsMap to turn the list of 
# tuples into a Dictionary too!